# MMOB Project

# import

in this cell, relevant libraries are imported, as well as the datafile imported from our github and a dict that will store all results during the project 

In [ ]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, Derive
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
data_file ='https://raw.githubusercontent.com/DunodMax/MMOB/main/Dataset.txt'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC
database = db.Database('LPMC', LPMC)
all_results = {}
from collections import namedtuple

In [ ]:
LPMC.head()

# Model 0

all useful columns for this model are used as variables

In [ ]:
travel_mode=Variable('travel_mode')
dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')
pt_interchanges=Variable('pt_interchanges')
dur_driving=Variable('dur_driving')
cost_transit=Variable('cost_transit')
cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
dur_walking=Variable('dur_walking')
dur_cycling=Variable('dur_cycling')
female=Variable('female')
age=Variable('age')

Trip duration for each mode are computed ( see report for details about equations)
Cost for each mode with a cost are computed ( cycling and walking is considered free)

In [ ]:
time_walk=dur_walking
time_cycle=dur_cycling
time_pt=dur_pt_access+dur_pt_rail+dur_pt_bus+dur_pt_int
time_drive=dur_driving

cost_drive=cost_driving_ccharge+cost_driving_fuel
cost_pt=cost_transit

There is 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [ ]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

utility functions

In [ ]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

In [ ]:
av = {1: 1, 2: 1, 3: 1, 4:1}

estimation of the parameters for Model 0, all relevant parameters are printed

In [ ]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

In [ ]:
res.compileEstimationResults(all_results)

In [ ]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

# Model 1 

### Time specification

We will try a specification where the the cost coeffecient is Generic and the time coefficient is alternative specific

Again 4 different travel mode, 4 utility functions and 3 alternative specific constant.
This time we have the generic parameter for cost and a 4 new specific parameter for travel time

In [ ]:
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)

New utility functions :

In [ ]:
v_walk_model1 = beta_time_walk * time_walk  
v_cycle_model1 = asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1 = asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1 = asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

Estimation of the parameters for Model 1, all relevant parameters are printed

In [ ]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model1_time_specification'
all_results['Model1_time_specification'] = biogeme_model1.estimate()
results_alt_spec_time = biogeme_model1.estimate()
results_alt_spec_time.getEstimatedParameters()

In [ ]:
res.compileEstimationResults(all_results)

### Cost specification

We compare with the model where we specify the price, to be sure to have the most interesting model for the future models. (walk and cycling don't have cost, so only 2 specific parameter for cost)

In [ ]:
beta_cost_drive = Beta('beta_cost_drive', 0, None, None, 0)
beta_cost_pt = Beta('beta_cost_pt', 0, None, None, 0)

New utility function

In [ ]:
v_walk_model1_spec_cost = beta_time * time_walk  
v_cycle_model1_spec_cost = asc_cycle + beta_time * time_cycle 
v_pt_model1_spec_cost = asc_pt + beta_time * time_pt + beta_cost_pt * cost_pt
v_drive_model1_spec_cost = asc_drive + beta_time * time_drive + beta_cost_drive * cost_drive

In [ ]:
V_model1_spec_cost = {1: v_walk_model1_spec_cost , 2: v_cycle_model1_spec_cost, 3: v_pt_model1_spec_cost, 4: v_drive_model1_spec_cost}
logprob_model1_spec_cost = models.loglogit(V_model1_spec_cost, av, travel_mode)
biogeme_model1_spec_cost = bio.BIOGEME(database, logprob_model1_spec_cost)
biogeme_model1_spec_cost.modelName = 'Model_1_cost_specifiaction'
all_results['Model1_cost_specification'] = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost.getEstimatedParameters()

In [ ]:
res.compileEstimationResults(all_results)

### Comparaison

we use log-likehood ratio as the model 0 is a reduction of the model 1

In [ ]:
old  = results_generic
new  = results_alt_spec_time
new.likelihood_ratio_test(old, 0.05) #level of the statistics for a level of significance of 5%?

# Model 2

In [ ]:
LPMC.columns

In [ ]:
model_base=biogeme_model1

In [ ]:
#we're adding the attribute distance to our model 
#we're adding the interaction between age and distance
pt_interchanges=Variable('pt_interchanges')
driving_license=Variable('driving_license')

In [ ]:
#defining associated coefficients
beta_pt_inter= Beta('beta_pt_inter', 0, None, None, 0)
beta_license= Beta('beta_license', 0, None, None, 0)

In [ ]:
#the specification, we only consider the alternative distance for walking
#the interaction distance age is considered just for public trasnport
v_walk_model2 = v_walk_model1
v_cycle_model2 = v_cycle_model1
v_pt_model2 = v_pt_model1 + beta_pt_inter*pt_interchanges
v_drive_model2 = v_drive_model1 + asc_drive*beta_license*driving_license

In [ ]:
#the estimation results
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model_2'] = biogeme_model2.estimate()
results_model2 = biogeme_model2.estimate()
results_model2.getEstimatedParameters()

In [ ]:
res.compileEstimationResults(all_results)

In [ ]:
#we test base_model against model_2 where base_model is a restricted version of model_2 suing the looglikehood
#ratio test

In [ ]:
#we test them using the null hypothesis that our base model is the true model

In [ ]:
results_base=model_base.estimate()

In [ ]:
results_model2.likelihood_ratio_test(results_base, 0.05) #level of the statistics for a level of significance of 5%?

In [ ]:
#then depending on if the value I obtained was higher or lower than a certain number, we reject or accept the null hypothesis

In [ ]:
from scipy.stats import chi2
threshold = chi2.ppf(.95, 2)
threshold #just in case we might want to compare to this threshold

# Model 3

We will conduct a non-linear transformation of time

In [ ]:
model_base=biogeme_model2
log_time_walk = database.DefineVariable('log_time_walk', log(time_walk))
log_time_cycle = database.DefineVariable('log_time_cycle', log(time_cycle))
log_time_pt = database.DefineVariable('log_time_pt', log(time_pt))
log_time_drive = database.DefineVariable('log_time_drive', log(time_drive))
log_beta_time_drive = Beta('log_beta_time_drive', 0, None, None, 0)
log_beta_time_pt = Beta('log_beta_time_pt', 0, None, None, 0)
log_beta_time_walk = Beta('log_beta_time_walk', 0, None, None, 0)
log_beta_time_cycle = Beta('log_beta_time_cycle', 0, None, None, 0)

In [ ]:
v_walk_model3 = log_beta_time_walk * log_time_walk
v_cycle_model3 = asc_cycle + log_beta_time_cycle * log_time_cycle
v_pt_model3 = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model3 = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [ ]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

In [ ]:
res.compileEstimationResults(all_results)

We will compare Model_2 and Model_3 with a cox test, thus we will create a generic model

In [ ]:
v_walk_model_generic = log_beta_time_walk * log_time_walk + beta_time_walk * time_walk
v_cycle_model_generic = asc_cycle + log_beta_time_cycle * log_time_cycle + beta_time_cycle * time_cycle
v_pt_model_generic = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges + beta_time_pt * time_pt
v_drive_model_generic = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license + beta_time_drive * time_drive

In [ ]:
V_model_generic = {1: v_walk_model_generic , 2: v_cycle_model_generic, 3: v_pt_model_generic, 4: v_drive_model_generic}
logprob_model_generic = models.loglogit(V_model_generic, av, travel_mode)
biogeme_model_generic = bio.BIOGEME(database, logprob_model_generic)
biogeme_model_generic.modelName = 'Model_generic'
all_results['Model_generic'] = biogeme_model_generic.estimate()
results_model_generic = biogeme_model_generic.estimate()
results_model_generic.getEstimatedParameters()

In [ ]:
res.compileEstimationResults(all_results)

In [ ]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

In [ ]:
model_base=biogeme_model3
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

In this case the cox-test tells us to find better models, we will thus use a box-cox transform

In [ ]:
lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0)
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)
v_walk_model_boxcox = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model_boxcox  = asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model_boxcox  = asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model_boxcox  = asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [ ]:
V_model_boxcox = {1: v_walk_model_boxcox , 2: v_cycle_model_boxcox, 3: v_pt_model_boxcox, 4: v_drive_model_boxcox}
logprob_model_boxcox = models.loglogit(V_model_boxcox, av, travel_mode)
biogeme_model_boxcox = bio.BIOGEME(database, logprob_model_boxcox)
biogeme_model_boxcox.modelName = 'Model_boxcox'
all_results['Model_boxcox'] = biogeme_model_boxcox.estimate()
results_model_boxcox = biogeme_model_boxcox.estimate()
results_model_boxcox.getEstimatedParameters()

In [ ]:
res.compileEstimationResults(all_results)

In [ ]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_boxcox.likelihood_ratio_test(results_base, 0.05)

# Model 4

In [ ]:

model_base=biogeme_model_boxcox
V_nested=V_model_boxcox
#create the nests
Mu_motorized=Beta("Mu_motorized",1,0,None,0)
Mu_notmotorized=Beta("Mu_notmotorized",1,0,None,0)
motorized= Mu_motorized,[3,4]
notmotorized=Mu_notmotorized, [1,2]
nests_motor=motorized, notmotorized


In [ ]:
logprob_model_nested = models.lognested(V_nested, av, nests_motor, travel_mode)
biogeme_model_nested = bio.BIOGEME(database, logprob_model_nested)
biogeme_model_nested.modelName = 'Model_nested_motor'
all_results['Model_nested_motor'] = biogeme_model_nested.estimate()
results_model_nested = biogeme_model_nested.estimate()
results_model_nested.getEstimatedParameters()

This model is rejected as Mu_notmotorized is inferior to 1. We will try a crossnested model between motored and private.

In [ ]:
#define the crossnests
Mu_private=Beta('Mu_private',3,1,None,0)
Mu_motorcross=Beta('Mu_Motorcross',3,1,None,0)
Alpha_Motor=Beta('Alpha_Motor',0.5,0,1,0)
Alpha_Private=1-Alpha_Motor

alpha_motor={1:0.0,
             2:0.0,
             3:Alpha_Motor,
             4:1.0 }
alpha_private={1:1.0,
               2:1.0,
               3:Alpha_Private,
               4:0.0}

nest_motor=Mu_motorcross, alpha_motor
nest_private=Mu_private, alpha_private
cross_nests=nest_private, nest_motor

In [ ]:
logprob_model_crossnested = models.logcnl_avail(V_nested, av, cross_nests, travel_mode)
logprob_model_crossnested.changeInitValues(results_model_nested.getBetaValues())
biogeme_model_crossnested = bio.BIOGEME(database, logprob_model_crossnested)
biogeme_model_crossnested.modelName = 'Model_crossnested'
results_model_crossnested = biogeme_model_crossnested.estimate()
all_results['Model_crossnested'] = results_model_crossnested
results_model_crossnested.getEstimatedParameters()

The model is also rejected as alpha is 1. Moreover we see that mu_private is 1. We try another simple nest with the use of a vehicule:

In [ ]:
Mu_vehicle=Beta("Mu_vehicle",1,0,None,0)
vehicle= Mu_vehicle,[2,3,4]
not_vehicle=1,[1]
nests_vehicle=vehicle, not_vehicle


In [ ]:
logprob_model_nestedvehicle = models.lognested(V_nested, av, nests_vehicle, travel_mode)
biogeme_model_nestedvehicle = bio.BIOGEME(database, logprob_model_nestedvehicle)
biogeme_model_nestedvehicle.modelName = 'Model_nested_vehicle'
results_model_nestedvehicle = biogeme_model_nestedvehicle.estimate()
all_results['Model_nested_vehicle'] = results_model_nestedvehicle
table_nestedvehicle=results_model_nestedvehicle.getEstimatedParameters()
table_nestedvehicle

In [ ]:
#t-test H0:true value is 1
mu = table_nestedvehicle.loc['Mu_vehicle', 'Value']
mu_stderr = table_nestedvehicle.loc['Mu_vehicle', 'Rob. Std err']
tested_value = 1
ttest = (mu - tested_value) / mu_stderr
ttest

In [ ]:
res.compileEstimationResults(all_results)

In [ ]:
results_base=model_base.estimate()
results_model_nestedvehicle.likelihood_ratio_test(results_base, 0.05)

In [ ]:
V_pref=V_nested
nest_pref=nests_vehicle
model_name_pref='Model_nested_vehicle'
result_pref=results_model_nestedvehicle
estimation_pref=table_nestedvehicle

### Market share

In [ ]:
LPMC.head()

In [ ]:
LPMC[(LPMC["household_id"]==5) & (LPMC["person_n"]==1)].shape[0]

Some individuals make several trips. To account for this in the computation of the weights, we'll consider an individual making n trips as n indivudals. 

In [ ]:
census = {
    'male_41_more':  1633263,
    'male_40_less':  2676249,
    'female_41_more':   1765143,
    'female_40_less':  2599058,
}

In [ ]:
total = sum(census.values())
total

In [ ]:
filters = {
    'male_41_more': (LPMC.age >= 41) & (LPMC.female == 0),
    'male_40_less': (LPMC.age <= 40) & (LPMC.female == 0),
    'female_41_more': (LPMC.age >= 41) & (LPMC.female == 1),
    'female_40_less': (LPMC.age <= 40) & (LPMC.female == 1),
}

In [ ]:
sample_segments = {
    k: v.sum() for k, v in filters.items()
}
sample_segments

In [ ]:
total_sample = sum(sample_segments.values())
total_sample

In [ ]:
weights = {
    k: census[k] * total_sample / (v * total) 
    for k, v in sample_segments.items()
}
weights

In [ ]:
for k, f in filters.items():
    LPMC.loc[f, 'weights'] = weights[k] 

In [ ]:
#we check the weights sum to the sample size
sum_weights = LPMC['weights'].sum()
sum_weights

# Market Shares


We will there use the market_share function defined in lab session 03-indicators, but we adapted it to our models


Here we DO NOT simulate weights ? is it ok ?

## Market Shares with Modelpref

In [ ]:
female=Variable('female')
age=Variable('age')
weights=Variable('weights')

In [ ]:
type(weights)
IndicatorTuple = namedtuple('IndicatorTuple', 'value lower upper') 

In [ ]:
def market_share(utility):
    """Calculate the market shares of all alternatives, given the
    specification of the utility functions.

    :param utility: specification of the utility functions. It is a
        dict where the keys are the IDs of the alternatives, and the
        values are the expressions of the utility functions.
    :type utility: dict(int: biogeme.expressions.Expression)

    :return: a dictionary where each entry corresponds to an
        alternative, and associates its name with the InticatorTuple
        containing the value of the market share, and the lower and
        upper bounds of the 90% confidence interval.
    :rtype: dict(str: IndicatorTuple)

    """
    prob_WALK = models.lognested(utility, av, nest_pref, 1)
    prob_CYCLE = models.lognested(utility, av, nest_pref, 2)
    prob_PT = models.lognested(utility, av, nest_pref, 3)
    prob_DRIVE = models.lognested(utility, av, nest_pref, 4)
    """
    prob_WALK = models.loglogit(utility, None, 1)
    prob_CYCLE= models.loglogit(utility, None, 2)
    prob_PT = models.loglogit(utility, None, 3)
    prob_DRIVE = models.loglogit(utility, None, 4)
    """

    simulate = {
        'Prob. walk': prob_WALK,
        'Prob. cycle': prob_CYCLE,
        'Prob. PT': prob_PT,
        'Prob. drive': prob_DRIVE,
        'weights' : weights
        
    }
    biosim = bio.BIOGEME(database, simulate)
    simulated_values = biosim.simulate(result_pref.getBetaValues())

    # We also calculate confidence intervals for the calculated quantities

    betas = biogeme_model_nestedvehicle.freeBetaNames()
    b = result_pref.getBetasForSensitivityAnalysis(betas,useBootstrap=False)
    left, right = biosim.confidenceIntervals(b, 0.9)

    # Market shares are calculated using the weighted mean of the individual probabilities
    #Since lognested outputs the logarithm of the wanted probability, we'll have to compose the occurences
    #of the proba by the exponential

    ## Alternative WALK

    simulated_values['Weighted prob. walk'] = simulated_values['weights'] * np.exp(simulated_values['Prob. walk'])
    left['Weighted prob. walk'] = left['weights'] * np.exp(left['Prob. walk'])
    right['Weighted prob. walk'] = right['weights'] * np.exp(right['Prob. walk'])

    marketShare_walk = simulated_values['Weighted prob. walk'].mean()
    marketShare_walk_left = left['Weighted prob. walk'].mean()
    marketShare_walk_right = right['Weighted prob. walk'].mean()
    
    
    ## Alternative cycle

    simulated_values['Weighted prob. cycle'] =simulated_values['weights'] * np.exp(simulated_values['Prob. cycle'])
    left['Weighted prob. cycle'] = left['weights'] * np.exp(left['Prob. cycle'])
    right['Weighted prob. cycle'] = right['weights'] * np.exp(right['Prob. cycle'])

    marketShare_cycle = simulated_values['Weighted prob. cycle'].mean()
    marketShare_cycle_left = left['Weighted prob. cycle'].mean()
    marketShare_cycle_right = right['Weighted prob. cycle'].mean()

    ## Alternative pt

    simulated_values['Weighted prob. PT'] = (
        simulated_values['weights'] * np.exp(simulated_values['Prob. PT'])
    )
    left['Weighted prob. PT'] = left['weights'] * np.exp(left['Prob. PT'])
    right['Weighted prob. PT'] = right['weights'] * np.exp(right['Prob. PT'])

    marketShare_PT = simulated_values['Weighted prob. PT'].mean()
    marketShare_PT_left = left['Weighted prob. PT'].mean()
    marketShare_PT_right = right['Weighted prob. PT'].mean()
    
                                                         
    ## Alternative drive

    simulated_values['Weighted prob. drive'] = (
        simulated_values['weights'] * np.exp(simulated_values['Prob. drive'])
    )
    left['Weighted prob. drive'] = left['weights'] * np.exp(left['Prob. drive'])
    right['Weighted prob. drive'] = right['weights'] * np.exp(right['Prob. drive'])

    marketShare_drive = simulated_values['Weighted prob. drive'].mean()
    marketShare_drive_left = left['Weighted prob. drive'].mean()
    marketShare_drive_right = right['Weighted prob. drive'].mean()

    return {
            'Walk': IndicatorTuple(
                value=marketShare_walk, 
                lower=marketShare_walk_left, 
                upper=marketShare_walk_right
            ),
            'Cycle': IndicatorTuple(
                value=marketShare_cycle, 
                lower=marketShare_cycle_left, 
                upper=marketShare_cycle_right
            ),
            'Public transportation': IndicatorTuple(
                value=marketShare_PT, 
                lower=marketShare_PT_left, 
                upper=marketShare_PT_right
            ),
            'Drive': IndicatorTuple(
                value=marketShare_drive, 
                lower=marketShare_drive_left, 
                upper=marketShare_drive_right
            )
    }

In [ ]:
ms = market_share(V_pref)
for k, v in ms.items():
    print(
        f'Market share for {k}: {100*v.value:.2f}% '
        f'[{100*v.lower:.2f}%, '
        f'{100*v.upper:.2f}%]'
    )

#we check it sums up to 100
total=0
for k, v in ms.items():
    total+=100*v.value
print('Total : ', total)

In [ ]:
count_walk=0
count_cycle=0
count_pt=0
count_drive=0
for c in LPMC['travel_mode']:
    if c==1:
        count_walk+=1
    if c==2:
        count_cycle+=1
    if c==3:
        count_pt+=1
    if c==4:
        count_drive+=1
print('Market share for walk ' +  str(count_walk/5000))
print('Market share for cycle ' +  str(count_cycle/5000))   
print('Market share for pt ' +  str(count_pt/5000))   
print('Market share for drive ' +  str(count_drive/5000))   

##   Forecasting

### Question 1

##### Scenario 1 : an increase of car cost by 15%. 
 We use the beta elapsed and lambda as defined in model 3. 

Only the utility function for the car changes. The others take the values of Model_pref

In [ ]:
v_walk_scenario1 = V_pref[1]
v_cycle_scenario1   = V_pref[2]
v_pt_scenario1   = V_pref[3]
v_drive_scenario1   = asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive * (1+0.15) + asc_drive*beta_license*driving_license


V_scenario1  = {1: v_walk_scenario1  , 2: v_cycle_scenario1 , 3: v_pt_scenario1 , 4: v_drive_scenario1 }

In [ ]:
ms1 = market_share(V_scenario1)
for k, v in ms1.items():
    print(
        f'Market share for {k}: {100*v.value:.2f}% '
        f'[{100*v.lower:.2f}%, '
        f'{100*v.upper:.2f}%]'
    )


#we check it sums up to 100
total=0
for k, v in ms1.items():
    total+=100*v.value
print('Total : ', total)

##### Scenario 2 : Decrease of public transport cost by 15%

In [ ]:
v_walk_scenario2 = V_pref[1]
v_cycle_scenario2   = V_pref[2]
v_pt_scenario2   = asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt * (1-0.15) + beta_pt_inter*pt_interchanges
v_drive_scenario2   = V_pref[4]


V_scenario2  = {1: v_walk_scenario2  , 2: v_cycle_scenario2 , 3: v_pt_scenario2 , 4: v_drive_scenario2 }

In [ ]:
ms2 = market_share(V_scenario2)
for k, v in ms2.items():
    print(
        f'Market share for {k}: {100*v.value:.2f}% '
        f'[{100*v.lower:.2f}%, '
        f'{100*v.upper:.2f}%]'
    )

#we check it sums up to 100
total=0
for k, v in ms2.items():
    total+=100*v.value
print('Total : ', total)

### Question 3 : public transportation total revenue

In [ ]:
def revenues(cost, utility):
    """Calculate the revenues of all alternatives, given the
    specification of the utility functions and the expression of the cost.

    :param cost: expression to calculate the cost of the public transportation.
    :type cost: biogeme.expressions.Expression

    :param utility: specification of the utility functions. It is a
        dict where the keys are the IDs of the alternatives, and the
        values are the expressions of the utility functions.
    :type utility: dict(int: biogeme.expressions.Expression)
    
    :return: tuple containing the value of the revenues, as well as
        the lower and upper bound of the 90% confidence interval.
    :rtype: IndicatorTuple    
    """
    prob_WALK = models.lognested(utility, av, nest_pref, 1)
    prob_CYCLE = models.lognested(utility, av, nest_pref, 2)
    prob_PT = models.lognested(utility, av, nest_pref, 3)
    prob_DRIVE = models.lognested(utility, av, nest_pref, 4)

    simulate = {
        'weight': weights,
        'Revenues PT': prob_PT * cost,
    }
    biosim = bio.BIOGEME(database, simulate)
    simulated_values = biosim.simulate(result_pref.getBetaValues())

    # We also calculate confidence intervals for the calculated quantities

    betas = biogeme_model_nestedvehicle.freeBetaNames()
    b = result_pref.getBetasForSensitivityAnalysis(betas,useBootstrap=False)
    left, right = biosim.confidenceIntervals(b, 0.9)

    # Revenues are calculated using the weighted mean of the individual quantities


    simulated_values['Weighted revenues PT'] = (
        simulated_values['weight'] * simulated_values['Revenues PT']
    )
    left['Weighted revenues PT'] = left['weight'] * left['Revenues PT']
    right['Weighted revenues PT'] = right['weight'] * right['Revenues PT']

    revenues_PT = simulated_values['Weighted revenues PT'].mean()
    revenues_PT_left = left['Weighted revenues PT'].mean()
    revenues_PT_right = right['Weighted revenues PT'].mean()


    return IndicatorTuple(
        value=revenues_PT, 
        lower=revenues_PT_left, 
        upper=revenues_PT_right
    )

Total Revenues without any changement

In [ ]:
revenue_0 = revenues(cost_pt,V_pref)
print(
        f'Revenue for Scenario 0: {revenue_0[0]:.3f} '
        f'[{revenue_0[1]:.2f}, '
        f'{revenue_0[2]:.2f}]'
    )

##### Scenario 1 : an increase of car cost by 15% 
The factor is 1 as the price for public transport doen't change

In [ ]:
cost_1=cost_pt*1
revenue_scenario_1 = revenues(cost_1,V_scenario1)
print(
        f'Revenue for Scenario 1: {revenue_scenario_1[0]:.3f} '
        f'[{revenue_scenario_1[1]:.2f}, '
        f'{revenue_scenario_1[2]:.2f}]'
    )


##### Scenario 2 : a decrease of public transport cost by 15% 
The factor is 0.85

In [ ]:
cost_2=cost_pt*0.85
revenue_scenario_2 = revenues(cost_2,V_scenario2)
print(
        f'Revenue for the Scenario 2: {revenue_scenario_2[0]:.2f} '
        f'[{revenue_scenario_2[1]:.2f}, '
        f'{revenue_scenario_2[2]:.2f}]'
    )

### Question 4 : the average value of time for car and public transportation.

Calculate the value of time i.e. the willingess to pay to save travel time.

Reminder : V_pref = [1]->V_walk ; [2]->V_cycle ; [3]->V_pt ; [4]->V_drive ; 

In [ ]:
vot_pt =(Derive(V_pref[3],'dur_pt_access')+ Derive(V_pref[3],'dur_pt_rail')+Derive(V_pref[3], 'dur_pt_bus')+Derive(V_pref[3], 'dur_pt_int') ) / Derive(V_pref[3], 'cost_transit')

In [ ]:
vot_car = Derive(V_pref[4], 'dur_driving') / (Derive(V_pref[4], 'cost_driving_ccharge')+ Derive(V_pref[4], 'cost_driving_fuel'))

In [ ]:
simulate = {
    'weights': weights,
    'WTP PT time': vot_pt,
    'WTP CAR time': vot_car,
    'cost_transit': cost_transit,
    'cost_driving_ccharge': cost_driving_ccharge,
    'cost_driving_fuel': cost_driving_fuel,

}
biosim = bio.BIOGEME(database, simulate)
simulated_vot_values = biosim.simulate(result_pref.getBetaValues())


If the cost is 0, we need to set the value to 0 

In [ ]:
simulated_vot_values.loc[simulated_vot_values.cost_transit == 0, 'WTP PT time'] = 0
simulated_vot_values.loc[simulated_vot_values.cost_driving_ccharge == 0, 'WTP CAR time'] = 0
simulated_vot_values.loc[simulated_vot_values.cost_driving_fuel == 0, 'WTP CAR time'] = 0

Plot a Histogram to verify the coherance (car)

In [ ]:
_ = simulated_vot_values[simulated_vot_values['WTP CAR time'] != 0]['WTP CAR time'].hist(bins=200)

Public transportation

In [ ]:
_ = simulated_vot_values[simulated_vot_values['WTP PT time'] != 0]['WTP PT time'].hist(bins=200)

In [ ]:
avg_vot_pt = (
    simulated_vot_values['WTP PT time'] * 
    simulated_vot_values['weights']
).sum() / simulated_vot_values['weights'].sum()
print(f'Average value of time for PT: {avg_vot_pt:.3g} CHF/hour')

In [ ]:
avg_vot_car = (
    simulated_vot_values['WTP CAR time'] * 
    simulated_vot_values['weights']
).sum() / simulated_vot_values['weights'].sum()
print(f'Average value of time for car: {avg_vot_car:.3g} CHF/hour')